In [ ]:
import pandas as pd
import json

In [ ]:
with open("India-Food-Delivery-Time-Prediction.txt", "r") as file:
    raw_text = file.read()

# Parse the text as JSON
data = json.loads(raw_text)

# Convert to DataFrame
df = pd.DataFrame(data)

In [ ]:

df.to_csv("food_delivery_data.csv", index=False)
df

In [ ]:
df.isnull().sum()

#### Since you can see the time_ordered column has many null values we cannot drop all the rows as it may affect the accuracy of the model so we should fill it strategically.
We’re using Time_Order_picked and an estimated median preparation time to impute/fill these missing values.

In [ ]:
# Time columns are currently stored as strings. To perform time calculations (like differences), we convert them to proper datetime objects.
# errors='coerce' means: If a value can’t be converted (e.g., blank or malformed), it becomes NaT (Not a Time = missing datetime).
df['Time_Order_picked'] = pd.to_datetime(df['Time_Order_picked'], format='%H:%M:%S', errors='coerce')
df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'], format='%H:%M:%S', errors='coerce')

In [ ]:
# .dt.total_seconds() converts the timedelta to seconds, and we divide by 60 to get minutes.
prep_time = (df['Time_Order_picked'] - df['Time_Orderd']).dt.total_seconds() / 60
# We take the median (middle value) of all calculated prep times
median_prep_time = prep_time.median()

In [ ]:
filtered_rows = df['Time_Orderd'].isna() & df['Time_Order_picked'].notna()
df.loc[filtered_rows, 'Time_Orderd'] = df.loc[filtered_rows, 'Time_Order_picked'] - pd.to_timedelta(median_prep_time, unit='m')

In [ ]:
df.isnull().sum()

In [ ]:
# Find every occurrence of the word conditions and replace it with nothing ('').
# regex=False ensures that the string 'conditions' is treated literally, not as a regular expression.
# Removes any leading or trailing whitespace from each value in the column.
df['Weatherconditions'] = df['Weatherconditions'].str.replace('conditions','',regex=False).str.strip()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
#doing the same for time taken column and converting it into numeric type
df['Time_taken(min)'] = df['Time_taken(min)'].str.replace('(min)', '', regex=False).str.strip()
df['Time_taken(min)'] = pd.to_numeric(df['Time_taken(min)'], errors='coerce')

In [ ]:
#changing the data type of columns to numeric
df['Delivery_person_Age'] = pd.to_numeric(df['Delivery_person_Age'], errors='coerce')
df['Delivery_person_Ratings'] = pd.to_numeric(df['Delivery_person_Ratings'], errors='coerce')
df['multiple_deliveries'] = pd.to_numeric(df['multiple_deliveries'], errors='coerce')

In [ ]:
#droping duplicates if any
df.drop_duplicates(inplace=True)

In [ ]:
df

In [ ]:
# Strip and lower-case categories
df['City'] = df['City'].str.strip().str.lower()
df['Road_traffic_density'] = df['Road_traffic_density'].str.strip().str.lower()
df['Weatherconditions'] = df['Weatherconditions'].str.strip().str.lower()
df['Type_of_order'] = df['Type_of_order'].str.strip().str.lower()
df['Festival'] = df['Festival'].str.strip().str.lower()

In [ ]:
df[['Delivery_person_Age', 'Time_taken(min)']].describe()

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
df['City'].value_counts(), df['Road_traffic_density'].value_counts(), df['Weatherconditions'].value_counts(), df['Type_of_order'].value_counts(), df['Festival'].value_counts()

In [ ]:
df['Type_of_vehicle'].value_counts()

### Came across many fake null values so decided to convert it so that i can deal with them

In [ ]:
import numpy as np
df.replace(['NaN', 'nan', 'null', 'Null', 'N/A', '', ' ', 'na', 'NA'], np.nan, inplace=True)

In [ ]:
df.isna().sum().sort_values(ascending=False)

Filling delivery person ratings by median citywise

In [ ]:
# df['Delivery_person_Ratings'] = df.groupby('City')['Delivery_person_Ratings'].transform(
#     lambda x: x.fillna(x.median())
# )
city_median = df.groupby('City')['Delivery_person_Ratings'].transform('median')
df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].fillna(city_median)
df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].median())

In [ ]:
df['Delivery_person_Ratings'].isna().sum()

In [ ]:
#ensuring there are no outliers
df = df[(df['Delivery_person_Ratings'] >= 1.0) & (df['Delivery_person_Ratings'] <= 5.0)]

In [ ]:
df['Delivery_person_Ratings'].value_counts().sort_index()

Filling city column with the most frequent city

In [ ]:
# Fill missing values in 'City' with the most frequent city (mode)
city_mode = df['City'].mode()[0]
df['City'] = df['City'].fillna(city_mode)
df['City'].isna().sum()

Filling delivery_person_age by the median value of the column grouped by city

In [ ]:
city_median = df.groupby('City')['Delivery_person_Age'].transform('median')

In [60]:

df.loc[:, 'Delivery_person_Age'] = df['Delivery_person_Age'].fillna(city_median)
df.loc[:, 'Delivery_person_Age'] = df['Delivery_person_Age'].fillna(df['Delivery_person_Age'].median())

Filling multiple deliveries by most frequent value i.e mode

In [ ]:
df['multiple_deliveries'].value_counts()

In [ ]:
df.loc[:,'multiple_deliveries'] = df['multiple_deliveries'].fillna(df['multiple_deliveries'].mode()[0]).astype(int)
df['multiple_deliveries'].isna().sum()

Filling Weather conditions by the mode value grouped by city type

In [ ]:
df['Weatherconditions'].value_counts()

In [66]:
df.loc[:, 'Weatherconditions'] = df.groupby('City')['Weatherconditions'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown')
)

Filling Traffic density by the city wise mode value and festival by no 

In [71]:
df.loc[:,'Road_traffic_density'] = df.groupby('City')['Road_traffic_density'].transform( 
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown')
)
df['Festival'].value_counts(dropna=False)
df.loc[:,'Festival'] = df['Festival'].fillna('no')

In [ ]:
#Checking for any remaining missing values
df.isna().sum().sort_values(ascending=False)

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weatherconditions              0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken(min)                0
dtype: int64